- Прочитайте главы 12-14 из книги [Spark: The Definitive Guide](https://analyticsdata24.files.wordpress.com/2020/02/spark-the-definitive-guide40www.bigdatabugs.com_.pdf#%5B%7B%22num%22%3A484%2C%22gen%22%3A0%7D%2C%7B%22name%22%3A%22XYZ%22%7D%2C72%2C792%2Cnull%5D).
- Прочитайте главу 2 из книги [Hadoop: The Definitive Guide](https://grut-computing.com/HadoopBook.pdf#%5B%7B%22num%22%3A348%2C%22gen%22%3A0%7D%2C%7B%22name%22%3A%22XYZ%22%7D%2Cnull%2C588.58502%2Cnull%5D).
- Выполните задания.

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Lab4").getOrCreate()
sc=spark.sparkContext

textFiles = sc.textFile('hdfs://localhost:9000//user/student/gutenberg/*')

textFiles.first()

22/12/22 23:28:50 WARN Utils: Your hostname, student-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/12/22 23:28:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/22 23:28:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/22 23:28:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/12/22 23:28:53 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


22/12/22 23:29:00 WARN BlockReaderFactory: I/O error constructing remote block reader.
java.nio.channels.ClosedByInterruptException
	at java.nio.channels.spi.AbstractInterruptibleChannel.end(AbstractInterruptibleChannel.java:202)
	at sun.nio.ch.SocketChannelImpl.connect(SocketChannelImpl.java:658)
	at org.apache.hadoop.net.SocketIOWithTimeout.connect(SocketIOWithTimeout.java:191)
	at org.apache.hadoop.net.NetUtils.connect(NetUtils.java:586)
	at org.apache.hadoop.hdfs.DFSClient.newConnectedPeer(DFSClient.java:3033)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.nextTcpPeer(BlockReaderFactory.java:829)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.getRemoteBlockReaderFromTcp(BlockReaderFactory.java:754)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.build(BlockReaderFactory.java:381)
	at org.apache.hadoop.hdfs.DFSInputStream.getBlockReader(DFSInputStream.java:755)
	at org.apache.hadoop.hdfs.DFSInputStream.blockSeekTo(DFSInputStream.java:685)
	at org.apac

'The Project Gutenberg eBook of Ulysses, by James Joyce'

№0: Откройте каталог "/home/student/Work/Labs/3.Hadoop/WordCount" и изучите его содержимое.

Здесь представлена реализация подсчета слов в текстовых документах по парадигме MapReduce для Apache Hadoop. В папке Input находятся текстовые файлы с входными данными. Они же загружены в файловую систему HDFS по адресу hdfs://localhost:9000//user/student/gutenberg. В папке Src представлен исходный код стадий map и reduce. В папке Scripts есть два скрипта: cluster_run.sh и local_run.sh. Первый выполняет подсчет слов на платформе Hadoop. Второй выполняет ту же задачу без Hadoop. В папку Output попадают результаты выполнения скриптов.

Код ниже подсчитывает количество вхождений слов в текстовые документы с помощью платформы Apache Spark. Сравните этот код с реализацией той же задачи для Apache Hadoop.

In [2]:
words = textFiles\
    .flatMap(lambda line: line.split(" "))

counts = words\
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda x, y: x + y)

counts.take(6)

[('Project', 185),
 ('of', 23950),
 ('Ulysses,', 2),
 ('James', 30),
 ('Joyce', 3),
 ('', 34559)]

№1: Найдите количество вхождений только тех слов, длина которых не менее 5 символов. Выведите их в порядке убывания.

[('which', 3637),
 ('their', 1691),
 ('there', 1224),
 ('other', 954),
 ('would', 884),
 ('these', 846)]

In [21]:
words = textFiles.flatMap(lambda line: line.split(" "))
    
counts = words\
    .map(lambda word: (word, 1))\
    .filter(lambda word: len(word[0]) > 5)\
    .reduceByKey(lambda x, y: x + y)\
    .map(lambda word: (word[1],word[0])).sortByKey(False)
    
counts.take(10)

[(641, 'between'),
 (600, 'through'),
 (579, 'because'),
 (566, '[Footnote:'),
 (463, 'little'),
 (419, 'should'),
 (381, 'shadow'),
 (372, 'without'),
 (358, 'Leonardo'),
 (354, 'before')]

№2: Найдите количество повторений последовательностей из двух слов в текстах. Выведите их в порядке убывания.

[(('of', 'the'), 6900),
 (('in', 'the'), 3501),
 (('to', 'the'), 2208),
 (('and', 'the'), 1637),
 (('on', 'the'), 1618),
 (('from', 'the'), 1231)]

In [20]:
words = textFiles.flatMap(lambda line: line.split("."))\
   .map(lambda line: line.strip().split(" "))\
   .flatMap(lambda words: (tuple(word) for word in zip(words, words[1: ])))

counts = words\
     .map(lambda word: (word, 1))\
     .reduceByKey(lambda x, y: x + y)\
     .map(lambda word: (word[1],word[0])).sortByKey(False)

counts.take(10)

[(10929, ('', '')),
 (6900, ('of', 'the')),
 (3501, ('in', 'the')),
 (2208, ('to', 'the')),
 (1637, ('and', 'the')),
 (1618, ('on', 'the')),
 (1231, ('from', 'the')),
 (1157, ('of', 'a')),
 (1068, ('by', 'the')),
 (849, ('it', 'is'))]

№3: Найдите самое длинное слово из тех, которые состоят только из буквенно-цифровых символов.

In [19]:
words = textFiles.flatMap(lambda line: line.split("-"))\
        .flatMap(lambda line: line.split(" "))\
        .flatMap(lambda line: line.split("—"))\


counts = words\
     .map(lambda word: (word, 1))\
     .reduceByKey(lambda x, y: x + y)\
     .map(lambda word: (word[0])).sortBy(lambda word: len(word) * -1)

counts.take(1)

['Nationalgymnasiummuseumsanatoriumandsuspensoriumsordinaryprivatdocentge']

№4: Для каждой буквы найдите количество уникальных слов в текстах, которые на нее начинаются. Приведите буквы к верхнему регистру, отсортируйте их в алфавитном порядке.

[('A', 4268), ('B', 4460), ('C', 6660), ('D', 4156), ('E', 2856), ('F', 3482)]

In [18]:
words  = textFiles\
    .filter(lambda word: len(word) > 0 and word[0].isalpha())\
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda x, y: x + y)

counts = words.keys()\
    .map(lambda word: (word[0].upper(), 1)) \
    .reduceByKey(lambda x, y: x + y)\
    .sortByKey()

counts.take(6)

[('A', 4943), ('B', 2990), ('C', 2675), ('D', 1843), ('E', 1405), ('F', 2148)]

№ 5: Откройте каталог "/home/student/Work/Labs/3.Hadoop/Weather" и изучите его содержимое. Здесь представлен код MapReduce для Apache Hadoop, который обрабатывает погодные данные [National Climatic Data Center](http://www.ncdc.noaa.gov/). Подробности про задачу, которую решает данная программа, прочитайте во второй главе [Hadoop: The Definitive Guide](https://grut-computing.com/HadoopBook.pdf).

Погодные данные загружены в файловую систему HDFS по адресу hdfs://localhost:9000/user/student/weather. Там присутствуют данные только за 2022 год. Реализуйте решение той же задачи с погодными данными в Apache Spark с помощью методов flatMap и reduceByKey.

In [ ]:
import re
import sys

weatherFiles = sc.wholeTextFiles('hdfs://localhost:9000/user/student/weather/*')

def mapper(text):
    ...

def reducer(a, b):
    ...
    
weatherFiles\
    .flatMap(mapper)\
    .reduceByKey(reducer)\
    .collect()

№6: Решите задачу из задания 5 с помощью SQL и методов DataFrame.

In [ ]:
from pyspark.sql.functions import col

weatherLines = sc.textFile('hdfs://localhost:9000/user/student/weather/*')
weatherColumns = ["year","temperature", "quality"]
weatherDF = weatherLines\
    .map(lambda val: (int(val[15:19]), int(val[87:92]), int(val[92:93])))\
    .toDF(weatherColumns)

weatherDF.createOrReplaceTempView("Weather")

sqlWay = spark.sql("""

""")

dataFrameWay = weatherDF\
    ...

print(sqlWay.rdd.map(lambda x: (x[0], x[1])).collect())
print(dataFrameWay.rdd.map(lambda x: (x[0], x[1])).collect())